<a href="https://colab.research.google.com/github/juliee235/Name-Entity-Recognition/blob/main/NER_Thai_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Name Entity recognition


In [1]:
# install simple transformer
!pip install -q simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.8/258.8 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6

In [2]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/sample_submission.csv
/content/baseline.ipynb
/content/tag_list.csv
/content/AIFORTHAI-LST20Corpus.tar.gz
/content/LST20 Annotation Guideline.pdf
/content/test.csv
/content/.config/config_sentinel
/content/.config/.last_update_check.json
/content/.config/.last_opt_in_prompt.yaml
/content/.config/active_config
/content/.config/gce
/content/.config/.last_survey_prompt.yaml
/content/.config/default_configs.db
/content/.config/configurations/config_default
/content/.config/logs/2024.03.08/14.25.13.131140.log
/content/.config/logs/2024.03.08/14.25.22.236168.log
/content/.config/logs/2024.03.08/14.25.33.282965.log
/content/.config/logs/2024.03.08/14.24.35.829836.log
/content/.config/logs/2024.03.08/14.25.02.228998.log
/content/.config/logs/2024.03.08/14.25.34.023448.log
/content/sample_data/README.md
/content/sample_data/anscombe.json
/content/sample_data/mnist_train_small.csv
/content/sample_data/california_housing_test.csv
/content/sample_data/california_housing_train.csv
/content

## Import Dataset(LST20Corpus, Download at :https://aiforthai.in.th/corpus.php)

In [5]:
!tar -xf /content/AIFORTHAI-LST20Corpus.tar.gz

In [6]:
from datasets import load_dataset
lst20 = load_dataset("lst20", data_dir="/content/LST20_Corpus")
lst20


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for lst20 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/lst20
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/63310 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5620 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 63310
    })
    validation: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5620
    })
    test: Dataset({
        features: ['id', 'fname', 'tokens', 'pos_tags', 'ner_tags', 'clause_tags'],
        num_rows: 5250
    })
})

In [7]:
train_df = pd.DataFrame(lst20['train'])
validation_df = pd.DataFrame(lst20['validation'])
train_df

,id,fname,tokens,pos_tags,ner_tags,clause_tags
0,0,T00126.txt,"[สุรยุทธ์, ยัน, ปฏิเสธ, ลงนาม, _, MOU, _, กับ,...","[0, 1, 1, 1, 2, 0, 2, 4, 0, 12, 1, 0]","[8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3]"
1,1,T00126.txt,"[1, _, กันยายน, _, 2550, _, 12:21, _, น.]","[8, 2, 0, 2, 8, 2, 8, 2, 10]","[3, 13, 13, 13, 23, 0, 3, 13, 23]","[1, 2, 2, 2, 2, 2, 2, 2, 3]"
2,2,T00126.txt,"[นายก, ฯ, _, ย้ำ, ไม่, ลงนาม, ใน, _, MOU, _, ร...","[0, 2, 2, 1, 12, 1, 4, 2, 0, 2, 4, 0, 3, 5, 6,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, ..."
3,3,T00126.txt,"[ยัน, ไทย, ไม่, ได้, ล้มเหลว, ถึง, ขั้น, นั้น,...","[1, 0, 12, 5, 1, 1, 0, 9, 1, 7, 1, 0, 9, 12, 1...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,4,T00126.txt,"[พร้อม, หนุน, กกต., จัด, เลือกตั้ง, ให้, บริสุ...","[3, 1, 0, 1, 1, 3, 1, 1]","[0, 0, 7, 0, 0, 0, 0, 0]","[1, 2, 2, 2, 3, 1, 2, 3]"
...,...,...,...,...,...,...
63305,6,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, หาก, การ, ยื่น...","[0, 0, 2, 1, 3, 2, 3, 7, 1, 0, 12, 1, 3, 5, 1,...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 3, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63306,7,T13136.txt,"[เมื่อ, ถาม, ว่า, _, ใน, ทาง, การเมือง, _, หาก...","[4, 1, 3, 2, 4, 0, 0, 2, 3, 0, 3, 1, 0, 6, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63307,8,T13136.txt,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, เรื่อง, กฎหมาย...","[0, 0, 2, 1, 3, 2, 0, 0, 5, 3, 1, 2, 7, 1, 1, ...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
63308,9,T13136.txt,"[หาก, เป็น, การ, ขอ, ถวาย, ฎีกา, ตาม, โบราณ, ร...","[3, 1, 7, 1, 1, 0, 4, 0, 0, 2, 5, 1, 0, 3, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 1, 2, ..."


In [8]:
# Select NER columns
train_df = train_df[['id', 'tokens', 'ner_tags']]
validation_df = validation_df[['id', 'tokens', 'ner_tags']]
train_df

,id,tokens,ner_tags
0,0,"[สุรยุทธ์, ยัน, ปฏิเสธ, ลงนาม, _, MOU, _, กับ,...","[8, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0]"
1,1,"[1, _, กันยายน, _, 2550, _, 12:21, _, น.]","[3, 13, 13, 13, 23, 0, 3, 13, 23]"
2,2,"[นายก, ฯ, _, ย้ำ, ไม่, ลงนาม, ใน, _, MOU, _, ร...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, ..."
3,3,"[ยัน, ไทย, ไม่, ได้, ล้มเหลว, ถึง, ขั้น, นั้น,...","[0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,4,"[พร้อม, หนุน, กกต., จัด, เลือกตั้ง, ให้, บริสุ...","[0, 0, 7, 0, 0, 0, 0, 0]"
...,...,...,...
63305,6,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, หาก, การ, ยื่น...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
63306,7,"[เมื่อ, ถาม, ว่า, _, ใน, ทาง, การเมือง, _, หาก...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
63307,8,"[นาย, ปณิธาน, _, กล่าว, ว่า, _, เรื่อง, กฎหมาย...","[10, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
63308,9,"[หาก, เป็น, การ, ขอ, ถวาย, ฎีกา, ตาม, โบราณ, ร...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, ..."


In [9]:
NER_TAGS = ["O","B_BRN","B_DES","B_DTM","B_LOC","B_MEA","B_NUM","B_ORG","B_PER","B_TRM","B_TTL","I_BRN","I_DES","I_DTM","I_LOC","I_MEA","I_NUM",        "I_ORG",        "I_PER",        "I_TRM",        "I_TTL",
        "E_BRN","E_DES","E_DTM","E_LOC","E_MEA","E_NUM","E_ORG","E_PER","E_TRM","E_TTL"]
print(NER_TAGS)

['O', 'B_BRN', 'B_DES', 'B_DTM', 'B_LOC', 'B_MEA', 'B_NUM', 'B_ORG', 'B_PER', 'B_TRM', 'B_TTL', 'I_BRN', 'I_DES', 'I_DTM', 'I_LOC', 'I_MEA', 'I_NUM', 'I_ORG', 'I_PER', 'I_TRM', 'I_TTL', 'E_BRN', 'E_DES', 'E_DTM', 'E_LOC', 'E_MEA', 'E_NUM', 'E_ORG', 'E_PER', 'E_TRM', 'E_TTL']


### Preprocess Data

In [10]:
# seperate each words in token to dataframe
def separate_df(df):
  data_df = pd.DataFrame()
  sentence_id = []
  words = []
  labels = []
  for sentence in range(len(df)):
    for token in range(len(df['tokens'][sentence])):
      sentence_id.append(sentence)
      words.append(df['tokens'][sentence][token])
      labels.append(NER_TAGS[df['ner_tags'][sentence][token]])
  return pd.DataFrame(
      {"sentence_id": sentence_id, "words": words, "labels": labels}
  )

In [11]:
train_df = separate_df(train_df)
eval_data = separate_df(validation_df )
train_df

,sentence_id,words,labels
0,0,สุรยุทธ์,B_PER
1,0,ยัน,O
2,0,ปฏิเสธ,O
3,0,ลงนาม,O
4,0,_,O
...,...,...,...
2714721,63309,การ,O
2714722,63309,ถอน,O
2714723,63309,รายชื่อ,O
2714724,63309,ด้วย,O


## Model Training

In [12]:
import logging
from simpletransformers.ner import NERModel, NERArgs
import torch

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [13]:


ner_args = NERArgs()
ner_args.train_batch_size = 128
ner_args.use_multiprocessing = True
ner_args.evaluate_during_training = True
ner_args.eval_batch_size = 128
ner_args.num_train_epochs = 4
ner_args.learning_rate = 1e-4

model = NERModel(
     "roberta",
     "xlm-roberta-base",
     args=ner_args, use_cuda=torch.cuda.is_available(), labels=NER_TAGS
)



config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

You are using a model of type xlm-roberta to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLMRobertaTokenizer'. 
The class this function is called from is 'RobertaTokenizerFast'.


In [14]:


model.train_model(train_df, eval_data=eval_data)



/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/495 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


...

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/44 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.p

(1980,
 defaultdict(list,
             {'global_step': [495, 990, 1485, 1980],
              'train_loss': [0.10604732483625412,
               0.08517679572105408,
               0.08147518336772919,
               0.05266180634498596],
              'eval_loss': [0.14096237244931134,
               0.12672835182059894,
               0.12813135964626615,
               0.12986333346502346],
              'precision': [0.7480928971012036,
               0.7749114164722151,
               0.7893092828860536,
               0.7935509241053873],
              'recall': [0.8053287102513801,
               0.8181486381677996,
               0.8185136183220038,
               0.8285961950818924],
              'f1_score': [0.7756563770185653,
               0.7959432768913252,
               0.8036462182803646,
               0.8106949962058654]}))

## Evaluation

In [15]:


result, model_outputs, preds_list = model.eval_model(eval_data)
print(result)



/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/44 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_LOC seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_PER seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: B_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: I_DTM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.p

{'eval_loss': 0.12986333346502346, 'precision': 0.7935509241053873, 'recall': 0.8285961950818924, 'f1_score': 0.8106949962058654}


## Select the best Model

In [16]:
ner_args = NERArgs()
ner_args.eval_batch_size = 128
ner_args.use_multiprocessing = True
ner_args.max_seq_length = 512
model1 = NERModel(
     "auto", "/content/outputs/best_model", args=ner_args, use_cuda=torch.cuda.is_available(), labels= NER_TAGS
)

## Test Submission

In [17]:
test_df = pd.read_csv('/content/test.csv')
test_df

,i,word
0,0,ใน
1,1,สมัย
2,2,หนึ่ง
3,3,_
4,4,พระพุทธเจ้า
...,...,...
65002,65002,ผู้
65003,65003,สำรวม
65004,65004,ดี
65005,65005,แล้ว


In [18]:


txt = []

for i in test_df['word']:
    txt.append(i)
print(txt[:30])



['ใน', 'สมัย', 'หนึ่ง', '_', 'พระพุทธเจ้า', 'ประทับ', 'อยู่', 'วัด', 'เชตวัน', '_', 'เมือง', 'สาวัตถี', '_', 'ทรง', 'ปรารภ', 'การ', 'ถวาย', 'บริขาร', 'ทุก', 'อย่าง', 'ของ', 'พ่อค้า', 'ชาวเมือง', 'คน', 'หนึ่ง', 'ได้', 'ตรัส', 'อดีต', 'นิทาน', 'มา']


In [19]:
def split_into_sentences(tokens, tokens_per_sentence):
    sentences = []
    for i in range(0, len(tokens), tokens_per_sentence):
        sentence = tokens[i:i+tokens_per_sentence]
        sentences.append(sentence)
    return sentences

test_tokens1 = split_into_sentences(txt, 257)
predictions1 = model1.predict(test_tokens1, False)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:


tag_df = pd.read_csv('/content/tag_list.csv')
tag_df



,tag,class
0,O,0
1,B_ORG,1
2,B_PER,2
3,B_LOC,3
4,B_MEA,4
5,I_DTM,5
6,I_ORG,6
7,E_ORG,7
8,I_PER,8
9,B_TTL,9


In [21]:


final_test_df1 = []

for i in predictions1[0]:
    for j in i:
        for k in j.values():
            result = tag_df[tag_df['tag'] == k]['class'].values[0]
            final_test_df1.append(result)

print(len(final_test_df1))
print(test_df[60:80])
print(final_test_df1[60:80])
print(set(final_test_df1))



65007
     i          word
60  60             3
61  61             _
62  62           ตัว
63  63             _
64  64           คือ
65  65             _
66  66           ลิง
67  67             _
68  68  สุนัขจิ้งจอก
69  69             _
70  70           และ
71  71           นาก
72  72             _
73  73         สัตว์
74  74          ทั้ง
75  75             _
76  76             4
77  77             _
78  78          เป็น
79  79         สัตว์
[14, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 14, 0, 0, 0]
{0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 17, 18, 27}


In [22]:
submit_df1 = pd.read_csv('/content/sample_submission.csv')

submit_df1

,i,pred
0,0,0.0
1,1,0.0
2,2,0.0
3,3,NaN
4,4,NaN
...,...,...
65002,65002,NaN
65003,65003,NaN
65004,65004,NaN
65005,65005,NaN


In [23]:


final_result1 = pd.DataFrame(final_test_df1)
submit_df1['pred'] = final_result1
submit_df1

,i,pred
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
65002,65002,0
65003,65003,0
65004,65004,0
65005,65005,0


In [24]:


submit_df1.to_csv('Ner_mick.csv', index=False)



In [ ]:
!zip -r /content/best_model.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/model.safetensors (deflated 31%)
  adding: content/outputs/tokenizer_config.json (deflated 76%)
  adding: content/outputs/checkpoint-495-epoch-1/ (stored 0%)
  adding: content/outputs/checkpoint-495-epoch-1/model.safetensors (deflated 31%)
  adding: content/outputs/checkpoint-495-epoch-1/tokenizer_config.json (deflated 76%)
  adding: content/outputs/checkpoint-495-epoch-1/model_args.json (deflated 61%)
  adding: content/outputs/checkpoint-495-epoch-1/optimizer.pt (deflated 70%)
  adding: content/outputs/checkpoint-495-epoch-1/tokenizer.json (deflated 76%)
  adding: content/outputs/checkpoint-495-epoch-1/training_args.bin (deflated 52%)
  adding: content/outputs/checkpoint-495-epoch-1/config.json (deflated 61%)
  adding: content/outputs/checkpoint-495-epoch-1/unigram.json (deflated 74%)
  adding: content/outputs/checkpoint-495-epoch-1/special_tokens_map.json (deflated 52%)
  adding: content/outputs/checkpoint-495-epoch-1/e